# Notebook used to create features and edit datatypes

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from pandas.plotting import scatter_matrix, lag_plot,autocorrelation_plot
#from statsmodels.graphics.tsaplots import plot_pacf
import pyarrow as pa
import pyarrow.parquet as pq
from datetime import datetime
import re
import string


/Users/pauldefusco/anaconda2/envs/py27/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
master = pd.read_parquet('master.parquet',engine='fastparquet')


In [3]:
master = master.infer_objects()

In [4]:
master['host_response_time'].unique()


array(['within an hour', 'within a day', 'within a few hours', None,
       'a few days or more'], dtype=object)

Transform the Host Response Time into MAx ResponseNumber of Hours:

In [5]:
lst = []
for i in master['host_response_time']:
    if i == None:
        lst.append(None)
    elif i =='within an hour':
        lst.append(1)
    elif i == 'within a day':
        lst.append(24)
    elif i == 'within a few hours':
        lst.append(4)
    elif i == 'a few days or more':
        lst.append(50)
if len(lst) == len(master['host_response_time']):
    print "Total counts match"
        

Total counts match


In [6]:
master['id'].groupby(by=master['host_response_time']).count()

host_response_time
a few days or more     17816
within a day          154542
within a few hours    236514
within an hour        429227
Name: id, dtype: int64

In [7]:
master['MaxResponseTimeHours'] = lst

Transform Host Response Rate into float

In [8]:
for i in master['host_response_rate'].head(3):
    print i

100%
100%
100%


In [9]:
master['host_response_rate'] = master['host_response_rate'].astype(str).map(lambda x: x.lstrip('%'))
master['host_response_rate'] = master['host_response_rate'].apply(pd.to_numeric, errors='coerce')
master['host_response_rate'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')

Transform Host Is Super Host into Binary (Float)

In [10]:
for i in master['host_is_superhost'].head(3):
    print i

f
f
f


In [11]:
lst =[]
for i in master['host_is_superhost']:
    if i == None:
        lst.append(None)
    elif i =='t':
        lst.append(1)
    elif i == 'f':
        lst.append(0)
if len(lst) == len(master['host_is_superhost']):
    print "Total counts match"

Total counts match


In [12]:
master['host_is_superhost'] = lst

Convert Host Since to Date Time

In [13]:
for i in master['host_since'].head(3):
    print i

2016-04-12
2016-04-12
2016-04-12


In [14]:
lst = []
for i in master['host_since']:
    if i == None:
        lst.append(None)
    elif i!= None:
        lst.append(datetime.strptime(i,'%Y-%m-%d'))
if len(lst) == len(master['host_since']):
    print "Total counts match"
    

Total counts match


In [15]:
master['host_since'] = lst


In [ ]:
master['cancellation_policy'].dtypes


Now that we are done transforming text features into the data type they should have, we create basic features from the remaining text columns

Creating Basic Features from Text Features

In [16]:
master1 = master.copy()


In [17]:
master_objcols = master1.select_dtypes(include='object')

In [ ]:
#for i in master_objcols.head(1):
  #  print type(i)

In [23]:
def create_txt_features(pdseries):

    textLength = []
    textWordsPerc = []
    textPuncPerc = []
    textDigitsPerc = []

    for i in pdseries:
        tokens = re.findall(r"[\w']+|[.,!?;]", i)
        textLength.append(len(tokens))

        if len(tokens)==0:
            textWordsPerc.append(0)
            textPuncPerc.append(0)
            textDigitsPerc.append(0)

        else:
            textWordsPerc.append(len(i.split())/float(len(tokens)))
            textPuncPerc.append(len(''.join(c for c in i if c in string.punctuation))/float(len(tokens)))
            textDigitsPerc.append(len(''.join(c for c in i if c in string.digits))/float(len(tokens)))

    print textLength, textWordsPerc, textPuncPerc, textDigitsPerc
    

In [24]:
for i in master_objcols:
    create_txt_features(master_objcols[i])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


TypeError: expected string or buffer